In [1]:
from sklearn.model_selection import train_test_split
from sklearn import ensemble
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from math import sqrt
cleaned=pd.read_csv("C:/Users/avakk/Downloads/primarysecondary.csv")
cleaned.drop(["state","GDP2013","GDP2012","GDP2011"],axis=1,inplace=True)

for index,row in cleaned.iterrows():
    cleaned.at[index,"GDP2018"]=row["GDP2018"].replace(",","")
    cleaned.at[index,"GDP2017"]=row["GDP2017"].replace(",","")
    cleaned.at[index,"GDP2016"]=row["GDP2016"].replace(",","")
    cleaned.at[index,"GDP2015"]=row["GDP2015"].replace(",","")
    cleaned.at[index,"GDP2014"]=row["GDP2014"].replace(",","")
    
tonumeric=["GDP2018","GDP2017","GDP2016","GDP2015","GDP2014"]

for val in tonumeric:
    cleaned[val]=pd.to_numeric(cleaned[val])

y=cleaned["price"]
x=cleaned.drop('price',axis=1)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [2]:
numeric=["odometer","year","GDP2018","GDP2017","GDP2016","GDP2015","GDP2014"]

for val in numeric:
    x_min=x_train[val].min()
    x_max=x_train[val].max()
    x_train[val]=((x_train[val]-x_min)/(x_max-x_min))
    x_test[val]=((x_test[val]-x_min)/(x_max-x_min))
    
ordinal_columns=["cylinders"]

for col in ordinal_columns:
     le = LabelEncoder()
     le.fit_transform(list(x_train[col].astype(str).values))
     x_train[col] = le.transform(list(x_train[col].astype(str).values))

for col in ordinal_columns:
    le.fit(list(x_test[col].astype(str).values))
    x_test[col] = le.transform(list(x_test[col].astype(str).values))

C:\Users\avakk\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\avakk\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\avakk\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pyda

In [3]:
x_cylmin=x_train["cylinders"].min()
x_cylmax=x_train["cylinders"].max()
x_train["cylinders"]=((x_train["cylinders"]-x_cylmin)/(x_cylmax-x_cylmin))
x_test["cylinders"]=((x_test["cylinders"]-x_cylmin)/(x_cylmax-x_cylmin))

C:\Users\avakk\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\avakk\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [4]:
xx_train=pd.get_dummies(x_train)
xx_test=pd.get_dummies(x_test)

missing_cols=set(xx_train.columns)-set(xx_test.columns)
for val in missing_cols:
    xx_test[val]=0
    
xx_test=xx_test[xx_train.columns]

In [5]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators=20, random_state=0,n_jobs=-1)
regressor.fit(xx_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=-1,
                      oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

In [6]:
y_pred_test = regressor.predict(xx_test)
y_pred_train = regressor.predict(xx_train)

# Metrics
from sklearn import metrics

print('rmse (Test):', np.sqrt(metrics.mean_squared_error(y_test, y_pred_test)))
print("r2 (Test):{}".format(regressor.score(xx_test, y_test)))

print('rmse (Train):', np.sqrt(metrics.mean_squared_error(y_train, y_pred_train)))
print("r2 (Train):{}".format(regressor.score(xx_train, y_train)))

rmse (Test): 0.2920791105494937
r2 (Test):0.8406953851824001
rmse (Train): 0.1281362404555088
r2 (Train):0.9694628828048758


In [8]:
import pickle
pickle.dump(regressor, open('RandomFRegPh2.pkl','wb'))

In [9]:
regressor=pickle.load(open('RandomFRegPh2.pkl', 'rb'))